In [45]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import tqdm.notebook
import math

In [57]:
games = pd.read_csv("35개 리뷰.csv")

In [58]:
games.columns

Index(['매출 순위', '평가', '게임번호', '리뷰'], dtype='object')

In [59]:
games = games[~games['리뷰'].isna()]#챗봇에 대답이 있는것만 남기게 끔 전처리

In [60]:
games.head()

,매출 순위,평가,게임번호,리뷰
0,Teenage Mutant Ninja Turtles: Shredder's Revenge,"Very Positive<br>92% of the 1,118 user reviews...",1361510,와 옛날 오락기로 닌자거북이 즐겼던 아재들에겐 향수와 재미를 다 잡음 다만 구매자가...
1,Teenage Mutant Ninja Turtles: Shredder's Revenge,"Very Positive<br>92% of the 1,118 user reviews...",1361510,코와붕가
2,Teenage Mutant Ninja Turtles: Shredder's Revenge,"Very Positive<br>92% of the 1,118 user reviews...",1361510,의외로 완성도 있는 액션에 놀랐다 제작자들이 액션게임에 깊은 조예를 가지고 있다는 ...
3,Teenage Mutant Ninja Turtles: Shredder's Revenge,"Very Positive<br>92% of the 1,118 user reviews...",1361510,년 전쯤에도 친구없어서 못했는데 지금도 친구없는 찐따라서 못하고 있음
4,Teenage Mutant Ninja Turtles: Shredder's Revenge,"Very Positive<br>92% of the 1,118 user reviews...",1361510,아 백수새끼들 님들 일하러안감 밤 시 출시한 게임인데 플탐 시간 넘는 새끼들은 뭐임...


In [61]:
games=games[:500]

In [62]:
%%time
from sentence_transformers import SentenceTransformer

#corpus = games['리뷰'].values.tolist() # 리뷰 임베딩

model = SentenceTransformer('jhgan/ko-sroberta-multitask')

model.encode(games.loc[0,'리뷰'])# 리뷰 데이터 전부 인코딩 (센텐스 버트 모델)


#임베딩 = 정규화(컴퓨터가 알아먹게끔)

Wall time: 14.2 s


array([ 9.05299783e-02,  1.22921519e-01, -7.90986121e-02,  1.61308497e-01,
       -8.38510618e-02,  2.02855691e-01, -7.89245516e-02,  3.60057533e-01,
       -6.50230388e-04,  4.69371676e-01, -4.35010642e-02, -2.19792992e-01,
       -2.56814837e-01, -2.66417146e-01,  1.61187202e-01,  2.76157737e-01,
        2.87277233e-02,  1.24222569e-01,  2.39824027e-01, -7.56515205e-01,
       -4.10361499e-01, -1.27363980e-01,  5.32586575e-01,  3.84001315e-01,
        2.33895883e-01, -8.42028320e-01, -7.71590993e-02,  3.82802412e-02,
        2.54592866e-01,  6.37688696e-01, -9.09822881e-02,  4.11095098e-02,
       -2.30129063e-01,  1.48268878e-01, -5.92473298e-02, -3.84750158e-01,
       -2.90026486e-01, -5.35245955e-01, -2.89914727e-01, -2.15051904e-01,
       -1.04931660e-01,  2.22441301e-01, -7.48177096e-02, -3.36907536e-01,
       -1.21996822e-02, -7.71923780e-01,  3.19533527e-01,  2.08518341e-01,
       -4.60207671e-01,  4.19869423e-02, -3.96433145e-01,  6.57716244e-02,
       -6.50890358e-03, -

In [63]:
# 유저 대화 내용 인코딩 전부 전부
#games['embedding'] = pd.Series( * len(games)) # 임베딩 칼럼 생성을 하는데 전처리한 데이터칸 만큼 늘림

games['embedding'] = games['리뷰'].map(lambda x: list(model.encode(x)))#판다스 프레임에 map으로 전체 적용

games

,매출 순위,평가,게임번호,리뷰,embedding
0,Teenage Mutant Ninja Turtles: Shredder's Revenge,"Very Positive<br>92% of the 1,118 user reviews...",1361510,와 옛날 오락기로 닌자거북이 즐겼던 아재들에겐 향수와 재미를 다 잡음 다만 구매자가...,"[0.09052998, 0.12292152, -0.07909861, 0.161308..."
1,Teenage Mutant Ninja Turtles: Shredder's Revenge,"Very Positive<br>92% of the 1,118 user reviews...",1361510,코와붕가,"[-0.6115216, -0.08164831, 0.3655754, -0.413465..."
2,Teenage Mutant Ninja Turtles: Shredder's Revenge,"Very Positive<br>92% of the 1,118 user reviews...",1361510,의외로 완성도 있는 액션에 놀랐다 제작자들이 액션게임에 깊은 조예를 가지고 있다는 ...,"[-0.41359878, 0.5680018, -0.4588233, 0.1335432..."
3,Teenage Mutant Ninja Turtles: Shredder's Revenge,"Very Positive<br>92% of the 1,118 user reviews...",1361510,년 전쯤에도 친구없어서 못했는데 지금도 친구없는 찐따라서 못하고 있음,"[-0.52376676, -0.062054068, -0.5288544, 0.0889..."
4,Teenage Mutant Ninja Turtles: Shredder's Revenge,"Very Positive<br>92% of the 1,118 user reviews...",1361510,아 백수새끼들 님들 일하러안감 밤 시 출시한 게임인데 플탐 시간 넘는 새끼들은 뭐임...,"[-0.5583926, 0.5802673, 0.62484866, 0.13071963..."
...,...,...,...,...,...
495,Battlefield™ 2042,"Mostly Negative<br>27% of the 106,019 user rev...",1517290,배필 사고 배필 하는 내 인생이 전드,"[-1.1380651, -0.18147941, -0.35470766, -0.4625..."
496,Battlefield™ 2042,"Mostly Negative<br>27% of the 106,019 user rev...",1517290,게임 사지마 만 천 만 천 만 주고 살만한 게임 아니다 이딴 배필 만든 다이스 새끼...,"[-0.52584136, 0.3148128, 0.18011883, 0.1387802..."
497,Battlefield™ 2042,"Mostly Negative<br>27% of the 106,019 user rev...",1517290,배틀필드는 에서 끝났다,"[0.1231938, 0.5045261, 1.1719387, -0.45820385,..."
498,Battlefield™ 2042,"Mostly Negative<br>27% of the 106,019 user rev...",1517290,배틀필드 분만 플레이하고 싶은,"[-0.5861346, 0.26623744, 1.0037533, -0.4796237..."


In [83]:
#games.to_csv('임베딩.csv',index =False) # 저장

In [84]:
#df = pd.read_csv('임베딩.csv') 

In [85]:
#df.info()

In [89]:
'''from sklearn.metrics.pairwise import cosine_similarity

input_emb=model.encode(input()) # ex ) 어려운 게임
games_sim = cosine_similarity([input_emb],[embeddings])
games_sim'''

'from sklearn.metrics.pairwise import cosine_similarity\n\ninput_emb=model.encode(input()) # ex ) 어려운 게임\ngames_sim = cosine_similarity([input_emb],[embeddings])\ngames_sim'

In [90]:
'''games_sim_sorted_ind = games_sim.argsort()[:, ::-1]
similar_indexes = games_sim_sorted_ind.reshape(-1)
recom_games=games.iloc[similar_indexes]


df = pd.DataFrame(recom_games[['매출 순위','평가']].drop_duplicates())
df[:10]'''

"games_sim_sorted_ind = games_sim.argsort()[:, ::-1]\nsimilar_indexes = games_sim_sorted_ind.reshape(-1)\nrecom_games=games.iloc[similar_indexes]\n\n\ndf = pd.DataFrame(recom_games[['매출 순위','평가']].drop_duplicates())\ndf[:10]"

In [103]:
임베딩 = model.encode(input())

환불 마렵다


In [104]:
games['distance'] = games['embedding'].map(lambda x: cosine_similarity([임베딩], [x]).squeeze())
# 사용자가 입력한 문장을 코사인 유사도로  유사성 분석

In [105]:
answer = games.loc[games['distance'].idxmax()]


In [106]:
print( answer['매출 순위'])
print(answer['리뷰'])
print(answer['distance'])

Battlefield™ 2042
환불 하고싶어요
0.7406724691390991
